In [1]:
using Circulation
using GPFields
using TimerOutputs

┌ Info: Precompiling Circulation [f9745a5e-b5bf-4554-ab79-8fb660046134]
└ @ Base loading.jl:1260


In [2]:
const DATA_DIR_BASE = let dir = expanduser("~/Dropbox/circulation/data")
    @show readdir(dir)
    # joinpath(dir, "4vortices")
    # joinpath(dir, "TG")
    joinpath(dir, "tangle")
end

const DATA_IDX = 1
const RESOLUTION = 256

const GP_PARAMS = ParamsGP(
    (RESOLUTION, RESOLUTION, RESOLUTION),
    c = 1.0,
    nxi = 1.5,
)

let dir = joinpath(DATA_DIR_BASE, string(RESOLUTION))
    println("Directories in $DATA_DIR_BASE: ", readdir(DATA_DIR_BASE))
    println("Files in $dir: ", readdir(dir))
end

readdir(dir) = ["4vortices", "KW", "TG", "TG2D", "rotation", "tangle"]
Directories in /home/jipolanco/Dropbox/circulation/data/tangle: ["128", "256"]
Files in /home/jipolanco/Dropbox/circulation/data/tangle/256: ["ImaPsi.001.dat", "ReaPsi.001.dat", "tt.001.dat"]


In [6]:
# loop_sizes = (1, 2, 4, 8, 16)
N = GP_PARAMS.dims[1]
# loop_sizes = (4, 8, )
power_max = Int(log2(N)) - 1
loop_sizes = [1 << n for n = 0:power_max]  # powers of two
@show loop_sizes
to = TimerOutput()

κ = GP_PARAMS.κ  # not sure if this is the good κ...

stats = Circulation.init_statistics(
    loop_sizes,
    hist_edges=LinRange(-10κ, 10κ, 500),
    num_moments=20,
)

@time analyse!(stats, GP_PARAMS, DATA_DIR_BASE,
    data_idx=DATA_IDX, eps_vel=0, to=to,
    slice_selection=1:16,  # to make things faster! (testing only)
);

loop_sizes = [1, 2, 4, 8, 16, 32, 64, 128]
 12.262821 seconds (17.39 k allocations: 18.932 MiB)


In [4]:
save_statistics("stats.h5", stats)

HFD5 data file (closed): stats.h5

In [5]:
println(to)

 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            20.4s / 71.7%            896MiB / 12.7%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 analyse!                   3    14.7s   100%   4.88s    114MiB  100%   37.9MiB
   circulation!         1.15k    5.59s  38.2%  4.85ms     0.00B  0.00%    0.00B
   statistics           1.15k    4.35s  29.7%  3.77ms   37.7KiB  0.03%    33.5B
     histogram          1.15k    2.60s  17.8%  2.26ms     0.00B  0.00%    0.00B
     moments            1.15k    1.74s  11.9%  1.51ms     0.00B  0.00%    0.00B
   load_psi!               48    1.07s  7.29%  22.2ms    557KiB  0.48%  11.6KiB
   prepare!               144    524ms 